In [1]:
import gym
from skgym.value_functions import GenericQ
from skgym.policies import ValuePolicy
from skgym.algorithms import QLearning
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import FunctionTransformer


# the Gym environment
env = gym.make('CartPole-v0')


# define sklearn model for approximating Q-function
regressor = SGDRegressor(eta0=0.1, learning_rate='constant')
transformer = FunctionTransformer(
    lambda x: np.hstack((x, x ** 2)), validate=False)


# define Q, its induced policy and update algorithm
Q = GenericQ(env, regressor, transformer)
policy = ValuePolicy(Q)
algo = QLearning(Q, gamma=0.8)


# number of iterations
num_episodes = 200
max_episode_steps = env._max_episode_steps


# used for early stopping
num_consecutive_successes = 0


for episode in range(1, num_episodes + 1):
    last_episode = episode == num_episodes or num_consecutive_successes == 9
    
    # init
    s = env.reset()
    
    # amount of random exploration
    if last_episode:
        epsilon = 0
        env.render()
    elif episode < 10:
        epsilon = 0.5
    else:
        epsilon = 0.01
    
    for t in range(1, max_episode_steps + 1):
        a = policy.epsilon_greedy(s, epsilon)
        s_next, r, done, info = env.step(a)
        
        # update or render
        if not last_episode:
            algo.update(s, a, r, s_next)            
        else:
            env.render()
        
        # keep track of consecutive successes
        if done:
            if t == max_episode_steps:
                num_consecutive_successes += 1
                print(f"num_consecutive_successes = {num_consecutive_successes}")
            else:
                num_consecutive_successes = 0
                print(f"failed after {t} steps")
            break
    
        # prepare for next step
        s = s_next

        
    if last_episode:
        break


env.close()

/home/kris/.local/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


failed after 37 steps
failed after 27 steps
failed after 39 steps
failed after 35 steps
failed after 19 steps
failed after 15 steps
failed after 14 steps
failed after 14 steps
failed after 9 steps
failed after 66 steps
failed after 8 steps
failed after 18 steps
failed after 9 steps
failed after 12 steps
failed after 10 steps
failed after 10 steps
failed after 9 steps
failed after 13 steps
failed after 8 steps
failed after 10 steps
failed after 10 steps
failed after 10 steps
failed after 9 steps
failed after 11 steps
failed after 13 steps
failed after 13 steps
failed after 9 steps
failed after 18 steps
failed after 19 steps
failed after 9 steps
failed after 15 steps
failed after 8 steps
failed after 19 steps
failed after 18 steps
failed after 18 steps
failed after 9 steps
failed after 10 steps
failed after 16 steps
failed after 20 steps
failed after 16 steps
failed after 32 steps
failed after 9 steps
failed after 31 steps
failed after 10 steps
failed after 31 steps
failed after 10 steps

In [2]:
# record video
env = gym.make('CartPole-v1')
env = gym.wrappers.Monitor(env, os.path.join('data', 'video', 'cartpole-linear-model-qlearning'), force=True)
s = env.reset()
env.render()
done = False

while not done:
    a = policy.greedy(s)
    s, _, done, _ = env.step(a)
    env.render()
    
env.close()

/home/kris/.local/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)
